# Challenge Exercise Solution


### This section contains the solution for the challenge, do not review this file without attempting the challenge



## Section 1: Classify the Documents

### Solution:

In [ ]:
import boto3
import requests
from textractprettyprinter.t_pretty_print import Textract_Pretty_Print, get_lines_string
import requests
import json
import os

In [ ]:
textract = boto3.client('textract')
comprehend = boto3.client('comprehend')

In [ ]:
documents = ['Image1.png', 'Image2.png','Image3.png','Image4.png','Image5.png']

<h3> Use this code block if your model isn't completed training </h3>

In [ ]:
#This code will help you classify your documents.
def classify(document):
    
    with open(document, 'rb') as document:
        imageBytes = bytearray(document.read())

    textract_response = textract.detect_document_text(Document={'Bytes': imageBytes})

    #Post-Process the response using Textract Response Parser 
    classification_text = get_lines_string(textract_json=textract_response)
    endpoint_url = "https://uctwj7mbwu.us-east-1.awsapprunner.com/classifydoc"

    payload = json.dumps({
      "classification_text": classification_text
    })
    headers = {
      'Content-Type': 'application/json'
    }
    response = requests.request("POST", endpoint_url, headers=headers, data=payload)
    return response.text

for doc in documents:
    print(doc + " is " + classify(doc))

#Once you have the right classes it's time to change the names of the finals to their appropriate names

In [ ]:
def classify(document):
    
    with open(document, 'rb') as document:
        imageBytes = bytearray(document.read())

    textract_response = textract.detect_document_text(Document={'Bytes': imageBytes})

    #Post-Process the response using Textract Response Parser 
    classification_text = get_lines_string(textract_json=textract_response)
    
    #send results to self-trained Comprehend Classifier, please insert your Model Endpoint ARN:
    endpoint_arn = "{enter_your_classification_model_arn}"
    comprehend_response = comprehend.classify_document(Text= classification_text,EndpointArn=endpoint_arn)
    predicted_class = comprehend_response['Classes'][0]['Name']
    #print(predicted_class)

    return predicted_class

for doc in documents:
    print(doc + " is " + classify(doc))


In [ ]:
#Once you have the right classes it's time to change the names of the finals to their appropriate names
os.rename('/root/reinventaim316/Module 5 - Challenge/Image1.png','/root/reinventaim316/Module 5 - Challenge/loan_app.png')
os.rename('/root/reinventaim316/Module 5 - Challenge/Image2.png','/root/reinventaim316/Module 5 - Challenge/mortgage_note.png')
os.rename('/root/reinventaim316/Module 5 - Challenge/Image3.png','/root/reinventaim316/Module 5 - Challenge/proof_of_identity.png')
os.rename('/root/reinventaim316/Module 5 - Challenge/Image4.png','/root/reinventaim316/Module 5 - Challenge/bank_statement.png')
os.rename('/root/reinventaim316/Module 5 - Challenge/Image5.png','/root/reinventaim316/Module 5 - Challenge/w2.png')

## Section 2: Document Extraction and Entity Extraction

### Document 1: Loan Application

In [ ]:
from trp import Document

In [ ]:
document = 'loan_app.png'

#Call the Analyze Doc API
with open(document, "rb") as document:
    analyze_result = textract.analyze_document(
        Document={
            'Bytes': document.read(),
        },
        FeatureTypes=["FORMS"])

#post-process and extract key values from textract response
doc = Document(analyze_result)

for page in doc.pages:

    # Search fields by key
    key = "PRIMARY BORROWER:"
    field = page.form.getFieldByKey(key)
    if(field):
        customer_name = str(field.value)
        print("Customer Name: " + customer_name)

    key = "DATE:"
    fields = page.form.searchFieldsByKey(key)
    for field in fields:
        application_date = str(field.value)
        print("Application Date: " + application_date)

    key = "CURRENT ADDRESS:"
    fields = page.form.searchFieldsByKey(key)
    for field in fields:
        address = str(field.value)
        
        #trim extra values from address:
        address = address[:len(address) - 32]

        print("Address: " + address)


## Document 2: Mortage Notes

In [ ]:
#We are getting the Entities from our Custom NER Model
input_file_path = "mortgage_note.png"
url = "https://uctwj7mbwu.us-east-1.awsapprunner.com/customner"

with open(input_file_path, "rb") as file:
    files = {
         "input_file": file
    }
    response = requests.post(url, files=files)
    responsecontent = response.json()
#set loan and payment variables from response
loan_amount = responsecontent['Principal_Amount']
monthly_payments = responsecontent['Monthly_Payment']

### Document 3: Proof of Identity

In [ ]:
document = 'proof_of_identity.png'

#Call the Analyze Doc API
with open(document, "rb") as document:
    analyze_result = textract.analyze_document(
        Document={
            'Bytes': document.read(),
        },
        FeatureTypes=["FORMS"])

#post-process and extract key values from textract response
doc = Document(analyze_result)

for page in doc.pages:

    # Search fields by key
    key = "LN FN"
    field = page.form.getFieldByKey(key)
    if(field):
        id_customer_name = str(field.value)
        print("Proof of ID Customer Name: " + id_customer_name)
        
#switch first and last name for comparison
names = str(id_customer_name).split(" ")
id_name = names[1] + " " + names[0]
        
#Compare customer name from ID to customer name on loan application
if str(customer_name) == id_name:
    identity_confirmed_boolean = True
else:
    identity_confirmed_boolean = False

print(identity_confirmed_boolean)


### Document 4: Bank Statement

In [ ]:
document = 'bank_statement.png'

#Call the Analyze Doc API
with open(document, "rb") as document:
    analyze_result = textract.analyze_document(
        Document={
            'Bytes': document.read(),
        },
        FeatureTypes=["FORMS"])

#post-process and extract key values from textract response
doc = Document(analyze_result)

for page in doc.pages:

    # Search fields by key
    key = "Ending Balance"
    field = page.form.getFieldByKey(key)
    if(field):
        bankstatement_amount = str(field.value)
        print("Bank Statement Amount: " + bankstatement_amount)


### Document 5: W2 Form

In [ ]:
document = 'w2.png'

#Call the Analyze Doc API
with open(document, "rb") as document:
    analyze_result = textract.analyze_document(
        Document={
            'Bytes': document.read(),
        },
        FeatureTypes=["FORMS"])

#post-process and extract key values from textract response
doc = Document(analyze_result)

for page in doc.pages:

    # Search fields by key
    key = "1 Wages, tips, other compensation"
    field = page.form.getFieldByKey(key)
    if(field):
        wages_amount = str(field.value)
        print("Wages Amount: " + wages_amount)
        
    key = "a Employee's social security number"
    field = page.form.getFieldByKey(key)
    if(field):
        ssn = str(field.value)
        print("SSN: " + ssn)


## Section 3: Verify Results

In [ ]:
url = 'https://uctwj7mbwu.us-east-1.awsapprunner.com/challenge'

#construct the json payload and make a request to the verification endpoint
headers = {'Content-type': 'application/json'}
json = {
    "customer_name":customer_name,
    "loan_amount": loan_amount,
    "monthly_payments": monthly_payments,
    "loan_application_date":application_date,
    "bankstatement_amount":bankstatement_amount,
    "social_security":ssn,
    "wages_w2":wages_amount,
    "address":address,
    "identity_confirmed_boolean":identity_confirmed_boolean
}

try:
    r = requests.post(url, headers=headers, json=json)
    print(r.text)
except requests.ConnectionError:
    print("Cannot connect to server")